## Acquisition Inference Unit Test

See ```tests.core.acquisition_test```

In [1]:
import os.path as osp
import pandas as pd
import celldom
from celldom.core import acquisition
from celldom.config import experiment_config

paths = [
    # APT 000
    # Group 1
    './chipML-day0-cohort40mbar/BF_ST_000_APT_000_20180101000000.tif',
    './chipML-day0-cohort40mbar/GFP_ST_000_APT_000_20180101002900.tif',
    './chipML-day0-cohort40mbar/CY5_ST_000_APT_000_20180101003000.tif',
    
    # Group 2
    './chipML-day0-cohort40mbar/BF_ST_000_APT_000_20180101013000.tif',
    './chipML-day0-cohort40mbar/GFP_ST_000_APT_000_20180101013000.tif',
    './chipML-day0-cohort40mbar/CY5_ST_000_APT_000_20180101013000.tif',
    
    # APT 001
    # Group 3
    './chipML-day0-cohort40mbar/BF_ST_000_APT_001_20180102000000.tif',
    './chipML-day0-cohort40mbar/GFP_ST_000_APT_001_20180102000001.tif',
    './chipML-day0-cohort40mbar/CY5_ST_000_APT_001_20180102000000.tif',
    
    # Group 4 (invalid -- missing channel in hour 1 group)
    './chipML-day0-cohort40mbar/BF_ST_000_APT_001_20180102010000.tif',
    './chipML-day0-cohort40mbar/GFP_ST_000_APT_001_20180102010001.tif',
    
    # Group 4 (invalid -- duplicate channel in hour 2 group)
    './chipML-day0-cohort40mbar/BF_ST_000_APT_001_20180102020000.tif',
    './chipML-day0-cohort40mbar/GFP_ST_000_APT_001_20180102020001.tif',
    './chipML-day0-cohort40mbar/CY5_ST_000_APT_001_20180102020100.tif',
    './chipML-day0-cohort40mbar/CY5_ST_000_APT_001_20180102020200.tif',
    
    # APT 002
    # Group 3 (invalid - missing channels)
    './chipML-day0-cohort40mbar/BF_ST_000_APT_002_20180102000000.tif'
]
config = experiment_config.ExperimentConfig(osp.join(celldom.test_data_dir, 'config', 'experiment', 'fluoro-expression.yaml'))

In [2]:
config.acquisition_channels

[{'content': 'cell', 'name': 'BF', 'type': 'brightfield'},
 {'content': 'expression', 'name': 'CY5', 'type': 'fluorescent'},
 {'content': 'expression', 'name': 'GFP', 'type': 'fluorescent'}]

In [3]:
df = acquisition.collapse_acquisitions(paths, config, min_gap_seconds=30*60, max_invalid_acquisitions=3, return_df=True)
df

acq  \
apartment chip cohort day street                                                        
000       ML   40mbar 0   000    0  <celldom.core.acquisition.Acquisition object a...   
                                 1  <celldom.core.acquisition.Acquisition object a...   
001       ML   40mbar 0   000    0  <celldom.core.acquisition.Acquisition object a...   
                                 1                                               None   
                                 2                                               None   
002       ML   40mbar 0   000    0                                               None   

                                               channels  \
apartment chip cohort day street                          
000       ML   40mbar 0   000    0       [BF, CY5, GFP]   
                                 1       [BF, CY5, GFP]   
001       ML   40mbar 0   000    0       [BF, CY5, GFP]   
                                 1            [BF, GFP]   
                                 2  [BF, CY5, CY5, GFP]   
002       ML   40mbar 0   000    0                 [BF]   

                                                                                paths  \
apartment chip cohort day street                                                        
000       ML   40mbar 0   000    0  [./chipML-day0-cohort40mbar/BF_ST_000_APT_000_...   
                                 1  [./chipML-day0-cohort40mbar/BF_ST_000_APT_000_...   
001       ML   40mbar 0   000    0  [./chipML-day0-cohort40mbar/BF_ST_000_APT_001_...   
                                 1  [./chipML-day0-cohort40mbar/BF_ST_000_APT_001_...   
                                 2  [./chipML-day0-cohort40mbar/BF_ST_000_APT_001_...   
002       ML   40mbar 0   000    0  [./chipML-day0-cohort40mbar/BF_ST_000_APT_002_...   

                                    valid  
apartment chip cohort day street           
000       ML   40mbar 0   000    0   True  
                                 1   True  
001       ML   40mbar 0   000    0   True  
                                 1  False  
                                 2  False  
002       ML   40mbar 0   000    0  False

In [4]:
(~df['valid']).sum()

3

### Check Real Dataset

In [5]:
files = !find /lab/data/celldom/dataset/dataset09/ | grep tif
df = acquisition.collapse_acquisitions(
    files, experiment_config.ExperimentConfig('/lab/repos/celldom/config/experiment/exp-20181121-ML-FL-Sample.yaml'),
    return_df=True
)
df.head()

acq  \
apartment cohort          day street                                                        
000       1000mbarsqueeze 0   000    0  <celldom.core.acquisition.Acquisition object a...   
                              006    0  <celldom.core.acquisition.Acquisition object a...   
                              012    0  <celldom.core.acquisition.Acquisition object a...   
                              018    0  <celldom.core.acquisition.Acquisition object a...   
                              024    0  <celldom.core.acquisition.Acquisition object a...   

                                              channels  \
apartment cohort          day street                     
000       1000mbarsqueeze 0   000    0  [BF, CY5, GFP]   
                              006    0  [BF, CY5, GFP]   
                              012    0  [BF, CY5, GFP]   
                              018    0  [BF, CY5, GFP]   
                              024    0  [BF, CY5, GFP]   

                                                                                    paths  \
apartment cohort          day street                                                        
000       1000mbarsqueeze 0   000    0  [/lab/data/celldom/dataset/dataset09/20181121_...   
                              006    0  [/lab/data/celldom/dataset/dataset09/20181121_...   
                              012    0  [/lab/data/celldom/dataset/dataset09/20181121_...   
                              018    0  [/lab/data/celldom/dataset/dataset09/20181121_...   
                              024    0  [/lab/data/celldom/dataset/dataset09/20181121_...   

                                        valid  
apartment cohort          day street           
000       1000mbarsqueeze 0   000    0   True  
                              006    0   True  
                              012    0   True  
                              018    0   True  
                              024    0   True

In [6]:
df['valid'].value_counts()

True    528
Name: valid, dtype: int64